In [2]:
from functools import reduce
import numpy as np
import scipy

import pyscf
from pyscf import fci
from pyscf import gto, scf, ao2mo, lo, tdscf, cc


def tda_denisty_matrix(td, state_id):
    '''
    Taking the TDA amplitudes as the CIS coefficients, calculate the density
    matrix (in AO basis) of the excited states
    '''
    cis_t1 = td.xy[state_id][0]
    dm_oo =-np.einsum('ia,ka->ik', cis_t1.conj(), cis_t1)
    dm_vv = np.einsum('ia,ic->ac', cis_t1, cis_t1.conj())

    # The ground state density matrix in mo_basis
    mf = td._scf
    dm = np.diag(mf.mo_occ)

    # Add CIS contribution
    nocc = cis_t1.shape[0]
    # Note that dm_oo and dm_vv correspond to spin-up contribution. "*2" to
    # include the spin-down contribution
    dm[:nocc,:nocc] += dm_oo * 2
    dm[nocc:,nocc:] += dm_vv * 2

    # Transform density matrix to AO basis
    mo = mf.mo_coeff
    dm = np.einsum('pi,ij,qj->pq', mo, dm, mo.conj())
    return dm

mol = gto.Mole()
mol.atom = '''
C          3.86480       -1.02720        1.27180
C          3.60170       -0.34020        0.07040
C          2.93000       -1.06360        2.30730
C          4.54410       -0.28950       -0.97720
C          2.32170        0.33090       -0.09430
C          3.20030       -1.73210        3.54060
C          1.65210       -0.39240        2.14450
C          4.27330        0.38740       -2.16700
C          2.05960        1.01870       -1.29550
C          1.37970        0.28170        0.95340
C          2.27260       -1.73270        4.55160
C          0.71620       -0.41900        3.22490
C          2.99590        1.05910       -2.32920
C          5.20900        0.41800       -3.24760
C          1.01870       -1.06880        4.39470
C          2.72630        1.73410       -3.55840
C          4.90670        1.07300       -4.41500
C          3.65410        1.73950       -4.56900
C         -3.86650        1.01870       -1.29550
C         -2.93020        1.05910       -2.32920
C         -3.60440        0.33090       -0.09430
C         -3.19980        1.73410       -3.55840
C         -1.65280        0.38740       -2.16700
C         -2.32430       -0.34020        0.07040
C         -4.54630        0.28170        0.95340
C         -2.27200        1.73950       -4.56900
C         -0.71710        0.41800       -3.24760
C         -1.38200       -0.28950       -0.97720
C         -2.06130       -1.02720        1.27180
C         -4.27400       -0.39240        2.14450
C         -1.01930        1.07300       -4.41500
C         -2.99610       -1.06360        2.30730
C         -5.20980       -0.41900        3.22490
C         -2.72580       -1.73210        3.54060
C         -4.90730       -1.06880        4.39470
C         -3.65350       -1.73270        4.55160
H          4.82300       -1.53290        1.39770
H          5.49910       -0.80290       -0.85660
H          4.15900       -2.23700        3.66390
H          1.10180        1.52560       -1.42170
H          0.42460        0.79440        0.83100
H          2.50000       -2.24040        5.48840
H         -0.23700        0.09640        3.10140
H          6.16210       -0.09790       -3.12730
H          0.29870       -1.07700        5.21470
H          1.76850        2.24120       -3.67870
H          5.62580        1.08320       -5.23570
H          3.42730        2.25190       -5.50340
H         -4.82430        1.52560       -1.42170
H         -4.15760        2.24120       -3.67870
H         -5.50150        0.79440        0.83100
H         -2.49880        2.25190       -5.50340
H          0.23610       -0.09790       -3.12730
H         -0.42700       -0.80290       -0.85660
H         -1.10300       -1.53290        1.39770
H         -0.30030        1.08320       -5.23570
H         -6.16310        0.09640        3.10140
H         -1.76710       -2.23700        3.66390
H         -5.62740       -1.07700        5.21470
H         -3.42610       -2.24040        5.48840
'''


mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

#mf = scf.ROHF(mol).x2c()
mf = scf.RHF(mol)
mf.verbose = 4
mf.get_init_guess(mol, key='minao')
mf.conv_tol = 1e-9
#mf.level_shift = .1
#mf.diis_start_cycle = 4
#mf.diis_space = 10
mf.run(max_cycle=200)


n_triplets = 2
n_singlets = 2

avg_rdm1 = mf.make_rdm1()












Initial guess from minao.


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/jm/nd85t1m57dv98qh_jtqr2clh0000gn/T/tmpdnsov0fw
max_memory 4000 MB (current use 0 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -1373.8028556562
  HOMO = -0.155360683388173  LUMO = -0.0462263741041608
cycle= 1 E= -1360.24317894862  delta_E= 13.6  |g|= 0.719  |ddm|= 11.3
  HOMO = -0.143564650973908  LUMO = 0.138616575609906
cycle= 2 E= -1360.45923594589  delta_E= -0.216  |g|= 0.214  |ddm|= 1.43
  HOMO = -0.16857409289756  LUMO = 0.12156954691401
cycle= 3 E= -1360.47370291225  delta_E= -0.0145  |g|= 0.126  |ddm|= 0.515
  HOMO = -0.161685188896432  LUMO = 0.130732383404331
cycle= 4 E= -1

In [3]:
# compute singlets
mytd = tdscf.TDA(mf)
mytd.singlet = True 
mytd = mytd.run(nstates=n_singlets)
mytd.analyze()
for i in range(mytd.nroots):
    avg_rdm1 += tda_denisty_matrix(mytd, i)



******** <class 'pyscf.tdscf.rhf.TDA'> for <class 'pyscf.scf.hf.RHF'> ********
nstates = 2 singlet
deg_eia_thresh = 1.000e-03
wfnsym = None
conv_tol = 1e-09
eigh lindep = 1e-12
eigh level_shift = 0
eigh max_space = 50
eigh max_cycle = 100
chkfile = /var/folders/jm/nd85t1m57dv98qh_jtqr2clh0000gn/T/tmpdnsov0fw
max_memory 4000 MB (current use 0 MB)


Excited State energies (eV)
[4.37757462 4.50864348]

** Singlet excitation energies and oscillator strengths **
Excited State   1:      4.37757 eV    283.23 nm  f=0.7064
     117 -> 123      -0.10590
     118 -> 124      -0.10581
     119 -> 122      -0.45930
     120 -> 121      -0.49153
Excited State   2:      4.50864 eV    274.99 nm  f=0.0000
     119 -> 121       0.47837
     120 -> 122       0.46727

** Transition electric dipole moments (AU) **
state          X           Y           Z        Dip. S.      Osc.
  1         2.3095     -1.1112      0.1355      6.5870      0.7064
  2        -0.0015      0.0006      0.0007      0.0000      

In [ ]:
# compute triplets 
mytd = tdscf.TDA(mf)
mytd.singlet = False 
mytd = mytd.run(nstates=n_triplets)
mytd.analyze()
for i in range(mytd.nroots):
    avg_rdm1 += tda_denisty_matrix(mytd, i)

# normalize
avg_rdm1 = avg_rdm1 / (n_singlets + n_triplets + 1)

In [ ]:
S = mf.get_ovlp()
F = mf.get_fock()
np.save("fock_mat18", F)
np.save("overlap_mat18", S)
np.save("density_mat18", mf.make_rdm1())
np.save("rhf_mo_coeffs18", mf.mo_coeff)
np.save("cis_sa_density_mat18", avg_rdm1)